# SPECTRE PROTOTYPE
---

## Dependencies

In [2]:
import sys
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras

from sklearn.preprocessing import StandardScaler, LabelBinarizer
from sklearn.decomposition import PCA

from confluent_kafka import Producer, Consumer, KafkaError
import requests

2023-05-28 00:46:26.332566: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-28 00:46:26.361116: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-28 00:46:26.361886: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-28 00:46:26.947957: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


## Load SavedModel

In [3]:
model = tf.saved_model.load("../spectre-ann/Model/DDOS_2/A/SavedModel")

2023-05-28 00:46:28.572742: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-05-28 00:46:28.573015: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1956] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


## Data Pipeline

### Data Preprocessing

In [9]:
def prod_dataprocess(csv_file_path):
    # Read the CSV file into a DataFrame
    df = pd.read_csv(csv_file_path)

    dimensions_num_for_PCA = 7
    
    def clean_dataset(df):
        assert isinstance(df, pd.DataFrame), "df needs to be a pd.DataFrame"
        df.dropna(inplace=True)
        indices_to_keep = ~df.isin([np.nan, np.inf, -np.inf]).any(axis=1)
        return df[indices_to_keep]

    def get_PCA_feature_names(num_of_pca_components):
        feature_names = []
        for i in range(num_of_pca_components):    
            feature_names.append(f"Principal component {i+1}")
        return feature_names
    
    # Renaming columns and creating a copy
    df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '')
    df_cleaned = df.copy()
    df_cleaned = clean_dataset(df_cleaned)

    # Resetting index and removing unneeded index column
    df_cleaned = df_cleaned.reset_index()
    df_cleaned.drop('index', axis=1, inplace=True)

    # Saving the label attribute before dropping it
    df_labels = df_cleaned['label']
    df_no_labels = df_cleaned.drop('label', axis=1, inplace=False)
    df_features = df_no_labels.columns.tolist()

    # Scaling the data
    df_scaled = StandardScaler().fit_transform(df_no_labels)
    df_scaled = pd.DataFrame(data=df_scaled, columns=df_features)

    # Performing PCA
    pca = PCA(n_components=dimensions_num_for_PCA)
    principal_components = pca.fit(df_no_labels).transform(df_no_labels)

    # Creating a DataFrame with principal components
    principal_component_headings = get_PCA_feature_names(dimensions_num_for_PCA)
    df_pc = pd.DataFrame(data=principal_components, columns=principal_component_headings)

    # Concatenating principal components and labels
    df_final = pd.concat([df_pc, df_labels], axis=1)

    # Applying LabelBinarizer to the labels
    lb = LabelBinarizer()
    df_final['label'] = lb.fit_transform(df_final['label'])

    # Displaying the final DataFrame
    #df_final
    #print(df_final)

    #return DataFrame
    return df_final

In [10]:
# Preprocess the CSV data
preprocessed_data = prod_dataprocess('../../dataset/CICIDS2017/MachineLearningCSV/MachineLearningCVE/Friday-WorkingHours-Afternoon-DDos.pcap_ISCX.csv')

### Kafka

In [15]:
import socket

# Check if Zookeeper is running on port 2181
sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
result = sock.connect_ex(('localhost', 2181))
if result == 0:
    print("Zookeeper is running on port 2181")
else:
    print("Zookeeper is not running on port 2181")

# Check if Kafka is running on port 9092
sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
result = sock.connect_ex(('localhost', 9092))
if result == 0:
    print("Kafka is running on port 9092")
else:
    print("Kafka is not running on port 9092")

Zookeeper is running on port 2181
Kafka is running on port 9092


In [18]:
# Create a Kafka producer
conf = {'bootstrap.servers': 'localhost:9092'}
producer = Producer(conf)

# Serialize the preprocessed DataFrame to a JSON string
produced_data = preprocessed_data.to_json(orient='records')

# Send the preprocessed DataFrame to the Kafka topic
producer.produce('preprocessed_data_topic', key='data_key', value=produced_data)
producer.flush()
producer.close()

KafkaException: KafkaError{code=MSG_SIZE_TOO_LARGE,val=10,str="Unable to produce message: Broker: Message size too large"}

%6|1685214602.336|FAIL|rdkafka#consumer-3| [thrd:127.0.0.1:9092/1]: 127.0.0.1:9092/1: Disconnected (after 484944ms in state UP)
%6|1685214602.336|FAIL|rdkafka#consumer-3| [thrd:127.0.0.1:9092/1]: 127.0.0.1:9092/1: Disconnected while requesting ApiVersion: might be caused by incorrect security.protocol configuration (connecting to a SSL listener?) or broker version is < 0.10 (see api.version.request) (after 0ms in state APIVERSION_QUERY)
%6|1685214602.336|FAIL|rdkafka#producer-7| [thrd:127.0.0.1:9092/1]: 127.0.0.1:9092/1: Disconnected while requesting ApiVersion: might be caused by incorrect security.protocol configuration (connecting to a SSL listener?) or broker version is < 0.10 (see api.version.request) (after 0ms in state APIVERSION_QUERY)
%6|1685214602.336|FAIL|rdkafka#consumer-3| [thrd:GroupCoordinator]: GroupCoordinator: 127.0.0.1:9092: Disconnected (after 0ms in state APIVERSION_QUERY, 1 identical error(s) suppressed)
%6|1685214602.337|FAIL|rdkafka#consumer-3| [thrd:localhost:9

In [13]:
from confluent_kafka import KafkaError

# Create a Kafka consumer
conf = {
    'bootstrap.servers': 'localhost:9092',
    'group.id': 'mygroup',
    'auto.offset.reset': 'earliest'
}
consumer = Consumer(conf)
consumer.subscribe(['preprocessed_data_topic'])

# Process the data from Kafka and make predictions
while True:
    msg = consumer.poll(1.0)

    if msg is None:
        # Wait for the message
        continue
    if msg.error():
        print("Consumer error: {}".format(msg.error()))
    else:
        # Deserialize the message value
        preprocessed_data = pd.read_json(msg.value(), orient='records')
        prediction = model.predict(preprocessed_data)

KeyboardInterrupt: 

%4|1685206649.792|MAXPOLL|rdkafka#consumer-3| [thrd:main]: Application maximum poll interval (300000ms) exceeded by 71ms (adjust max.poll.interval.ms for long-running message processing): leaving group
%6|1685210039.846|FAIL|rdkafka#consumer-3| [thrd:GroupCoordinator]: GroupCoordinator: 127.0.0.1:9092: Disconnected (after 389394ms in state UP, 1 identical error(s) suppressed)
%6|1685210039.846|FAIL|rdkafka#producer-2| [thrd:127.0.0.1:9092/1]: 127.0.0.1:9092/1: Disconnected while requesting ApiVersion: might be caused by incorrect security.protocol configuration (connecting to a SSL listener?) or broker version is < 0.10 (see api.version.request) (after 0ms in state APIVERSION_QUERY)
%6|1685210039.846|FAIL|rdkafka#consumer-3| [thrd:localhost:9092/bootstrap]: localhost:9092/bootstrap: Disconnected while requesting ApiVersion: might be caused by incorrect security.protocol configuration (connecting to a SSL listener?) or broker version is < 0.10 (see api.version.request) (after 0ms in sta

## Predictions

In [ ]:
# Process the data from Kafka and make predictions
for message in consumer:
    preprocessed_data = message.value
    prediction = model.predict(preprocessed_data)
    # Process the prediction and update the detection rate